In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/PhD/Linear regression/training_data.csv')
df

,word,speaker,emotion,time,position,target sentence,speaker gender,length,surprisal GPT,surprisal BERT,log probability,fold,baseline,gpt model,bert model
0,imamo,1,0,0.6,b,25,f,5.0,19.857792,3.380102e+00,-6.051349,7,0.394550,0.387825,0.405421
1,još,1,0,0.1,m,25,f,3.0,31.651057,1.627048e-04,-7.325636,7,0.289565,0.299861,0.281831
2,četiri,1,0,0.2,e,25,f,6.0,50.446848,1.993417e-04,-8.268886,7,0.474390,0.488627,0.451815
3,minuta,1,0,0.8,e,25,f,6.0,22.568617,5.535464e-04,-9.569658,7,0.486852,0.477997,0.464540
4,vremena do,1,0,0.3,e,25,f,9.0,32.872915,4.718853e-04,-13.097782,7,0.696440,0.684134,0.659803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75781,donese,1052,4,0.2,e,38,m,6.0,21.093443,1.265704e+00,-11.516400,1,0.509190,0.504188,0.493572
75782,odluku,1052,4,0.8,e,38,m,6.0,33.347344,3.223444e-03,-8.857048,1,0.479364,0.479375,0.448136
75783,o,1052,4,0.1,e,38,m,1.0,7.940099,5.159480e-07,-6.484565,1,0.174174,0.173329,0.176083
75784,njegovoj,1052,4,0.4,e,38,m,8.0,54.498621,2.224697e-02,-6.630206,1,0.565821,0.570743,0.521318


In [22]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [23]:
mse_list = []
results_df = pd.DataFrame(columns = df.columns.tolist().append('gpt model'))

for fold in df['fold'].unique():

    test_data = df[df['fold'] == fold]
    y_test = df[df['fold'] == fold][['time']]

    train_data = df[df['fold'] != fold]
    y_train = df[df['fold'] != fold][['time']]

    model = LinearRegression()
    model.fit(train_data[['length', 'log probability', 'surprisal GPT']], y_train)

    y_pred = model.predict(test_data[['length', 'log probability', 'surprisal GPT']])

    mse = mean_squared_error(y_test, y_pred)
    mse_list.append(mse)

    test_data.loc[:, 'gpt model'] = y_pred
    # Concatenate the DataFrames along rows (axis=0)
    results_df = pd.concat([results_df, test_data], axis=0)


# Calculate the average of mse_list
average_mse = sum(mse_list) / len(mse_list)

<ipython-input-23-ee6aaca6eaf3>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.loc[:, 'gpt model'] = y_pred
<ipython-input-23-ee6aaca6eaf3>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.loc[:, 'gpt model'] = y_pred
<ipython-input-23-ee6aaca6eaf3>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

In [24]:
for f, mse in zip(df['fold'].unique(),mse_list):
  print(f"Fold: {f}  MSE: {mse}")
print(f"Average mse over folds: {average_mse}")

Fold: 7  MSE: 0.07356992861829717
Fold: 9  MSE: 0.05524465044478194
Fold: 6  MSE: 0.06232191704964151
Fold: 3  MSE: 0.0656925959604444
Fold: 5  MSE: 0.05942738606202532
Fold: 4  MSE: 0.0614394892471634
Fold: 2  MSE: 0.07819449292473825
Fold: 8  MSE: 0.06648669707307678
Fold: 0  MSE: 0.052377457207019695
Fold: 1  MSE: 0.05389656613751882
Average mse over folds: 0.06286511807247072


In [25]:
mse_list = []
df = results_df
results_df = pd.DataFrame(columns = df.columns.tolist().append('bert model'))

for fold in df['fold'].unique():

    test_data = df[df['fold'] == fold]
    y_test = df[df['fold'] == fold][['time']]

    train_data = df[df['fold'] != fold]
    y_train = df[df['fold'] != fold][['time']]

    model = LinearRegression()
    model.fit(train_data[['length', 'log probability', 'surprisal BERT']], y_train)

    y_pred = model.predict(test_data[['length', 'log probability', 'surprisal BERT']])

    mse = mean_squared_error(y_test, y_pred)
    mse_list.append(mse)

    test_data.loc[:, 'bert model'] = y_pred
    # Concatenate the DataFrames along rows (axis=0)
    results_df = pd.concat([results_df, test_data], axis=0)


# Calculate the average of mse_list
average_mse = sum(mse_list) / len(mse_list)

<ipython-input-25-f992f869fad0>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.loc[:, 'bert model'] = y_pred
<ipython-input-25-f992f869fad0>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.loc[:, 'bert model'] = y_pred
<ipython-input-25-f992f869fad0>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [26]:
for f, mse in zip(df['fold'].unique(),mse_list):
  print(f"Fold: {f}  MSE: {mse}")
print(f"Average mse over folds: {average_mse}")

Fold: 7  MSE: 0.067935500349027
Fold: 9  MSE: 0.05283013357182645
Fold: 6  MSE: 0.06201635060357114
Fold: 3  MSE: 0.0644716574692065
Fold: 5  MSE: 0.05797049313239967
Fold: 4  MSE: 0.0610640396567946
Fold: 2  MSE: 0.07375895645905112
Fold: 8  MSE: 0.06436014494488683
Fold: 0  MSE: 0.0589340011118901
Fold: 1  MSE: 0.055452351796015445
Average mse over folds: 0.061879362909466884


In [27]:
results_df

,word,speaker,emotion,time,position,target sentence,speaker gender,length,surprisal GPT,surprisal BERT,log probability,fold,baseline,gpt model,bert model
0,imamo,1,0,0.6,b,25,f,5.0,19.857792,3.380102e+00,-6.051349,7,0.394550,0.387825,0.405421
1,još,1,0,0.1,m,25,f,3.0,31.651057,1.627048e-04,-7.325636,7,0.289565,0.299861,0.281831
2,četiri,1,0,0.2,e,25,f,6.0,50.446848,1.993417e-04,-8.268886,7,0.474390,0.488627,0.451815
3,minuta,1,0,0.8,e,25,f,6.0,22.568617,5.535464e-04,-9.569658,7,0.486852,0.477997,0.464540
4,vremena do,1,0,0.3,e,25,f,9.0,32.872915,4.718853e-04,-13.097782,7,0.696440,0.684134,0.659803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75781,donese,1052,4,0.2,e,38,m,6.0,21.093443,1.265704e+00,-11.516400,1,0.509190,0.504188,0.493572
75782,odluku,1052,4,0.8,e,38,m,6.0,33.347344,3.223444e-03,-8.857048,1,0.479364,0.479375,0.448136
75783,o,1052,4,0.1,e,38,m,1.0,7.940099,5.159480e-07,-6.484565,1,0.174174,0.173329,0.176083
75784,njegovoj,1052,4,0.4,e,38,m,8.0,54.498621,2.224697e-02,-6.630206,1,0.565821,0.570743,0.521318


In [28]:
results_df.to_csv('/content/drive/MyDrive/PhD/Linear regression/training_data.csv', index=False)